In [1]:
import sys
!{sys.executable} -m pip install opencv-python
!{sys.executable} -m conda install -c conda-forge dlib -y
!{sys.executable} -m pip install face_recognition
!{sys.executable} -m pip install --upgrade imutils

Solving environment: done

# All requested packages already installed.



In [8]:
from imutils import paths
import face_recognition
import pickle
import cv2
import os
import time

In [9]:
def learn_faces():
    image_paths = list(paths.list_images('faces'))
    known_encodings = []
    known_names = []
    # loop over the image paths
    for (i, image_path) in enumerate(image_paths):
        # extract the name from the image path
        name = image_path.split(os.path.sep)[-2]
        # load the input image and convert it from BGR
        # RGB for dlib usage
        image = cv2.imread(image_path)
        # standardize input image sizes
        cv2.resize(image, (100, 100), interpolation=cv2.INTER_LINEAR)
        rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

        #Use face_recognition to locate faces
        boxes = face_recognition.face_locations(rgb,model='hog')
        
        # compute the facial embedding for the face
        encodings = face_recognition.face_encodings(rgb, boxes)
        for encoding in encodings:
            known_encodings.append(encoding)
            known_names.append(name)
            
    #save encodings along with their names in dictionary data
    data = {"encodings": known_encodings, "names": known_names}
    
    #use pickle to save data into a file for later use
    f = open("face_enc", "wb")
    f.write(pickle.dumps(data))
    f.close()

In [ ]:
def recognize_face(image):

    # load the default haarcascade
    faceCascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
    
    # load the known faces and embeddings
    data = pickle.loads(open('face_enc', "rb").read())

    image = cv2.imread(image)
    if image is None:
        print("Invalid Image")
        quit()
    rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    
    # convert image to greyscale for haarcascade
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    
    # in my classifier, smoothing resulted in worse results so it is unused here
    smooth = cv2.GaussianBlur(gray, (125,125), 0)
    faces = faceCascade.detectMultiScale(gray,
                                         scaleFactor=1.1,
                                         minNeighbors=6,
                                         minSize=(60, 60),
                                         flags=cv2.CASCADE_SCALE_IMAGE)

    # the facial embeddings for face in input
    encodings = face_recognition.face_encodings(rgb)
    names = []
    # loop over the facial embeddings incase
    # we have multiple embeddings for multiple fcaes
    for encoding in encodings:
        # compare encodings with encodings in data["encodings"]
        # matches contain array with boolean values and True for the embeddings it matches closely
        # and False for rest
        matches = face_recognition.compare_faces(data["encodings"], encoding, 0.5)

        name = "Unknown"
        # check for matches in data[]
        if True in matches:
            # find indices of True
            matched_ids = [i for (i, b) in enumerate(matches) if b]
            counts = {}
            # loop over the matched indicies increment our matches
            for i in matched_ids:
                # check names
                name = data["names"][i]
                #increase count for the name we got
                counts[name] = counts.get(name, 0) + 1
                #set name which has highest count
                name = max(counts, key=counts.get)
                
                # note, this code is intended for multiple faces in our dictionary. I only have Kanye
                # I found it difficult to properly train my classifier so I stuck with one person
                # to avoid excessive false positives. however, the code/game is meant to be scaled


            # update the list of names
            names.append(name)
            # loop over the recognized faces
            for ((x, y, w, h), name) in zip(faces, names):
                # rescale the face coordinates
                # draw the predicted face name on the image
                cv2.rectangle(image, (x, y), (x + w, y + h), (255, 255, 0), 2)
                cv2.putText(image, name, (x, y), cv2.FONT_HERSHEY_DUPLEX, 0.75, (255, 255, 0), 1)
                
    # cv2.imshow("output", image)
    # This above line pops up a window with the output. 
    cv2.imwrite("./output.jpg", image)
    #cv2.waitKey(0)
    return

In [16]:
def main():
    learn_faces()
    recognize_face("KanyeLookAlike.jpg")
    # to use, uncomment above line and replace sample_image with your intended jpeg.
if __name__ == "__main__":
    main()

In [ ]:
#    x = input("Is there an update to the faces folder? (y/n)")
#    if x == "y":
#        learn_faces()
#    recognize_face()